In [41]:
# # Import packages

import time

import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select
from tqdm import tqdm
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

# # STATICS
VENDOR_URL = 'https://www.synology.com/en-global/support/download/'
SELECTOR_PRODUCT_TYPE = 'div.margin_bottom20 > select:nth-child(1)'
SELECTOR_PRODUCT = '//*[@id="heading_bg"]/div/div/div[2]/select'
SELECTOR_MD5 = '//*[@id="results"]/div[3]/div[2]/div[1]/div/div[1]/div[1]/div/div/div[3]/div/div/div/div[2]/div[2]/div/a'
SELECTOR_NEWEST_OS = '//*[@id="results"]/div[3]/div[2]/div[1]/div/div[1]/div[1]/div/div/div[1]'
SELECTOR_DOWNLOAD = '//*[@id="results"]/div[3]/div[2]/div[1]/div/div[1]/div[1]/div/div/div[3]/div/div/div/div[1]/a'
DOWNLOAD_PATH = 'data/'

# Selenium Webdriver Options, Download Path, Headless, Screensize, Webbrowser Version
options = Options()
options.headless = False

options.add_experimental_option("prefs", {
    "download.default_directory": rf"{DOWNLOAD_PATH}"
})

# # Initialize Chrome and open Vendor Website


class Synology_scraper:

    def __init__(
        self,
        url: str = VENDOR_URL,
        headless: bool = False,
        options: Options = options,
        max_products: int = float('inf')
    ):
        self.headless = headless
        self.url = url
        self.max_products = max_products
        self.driver = webdriver.Chrome(service=Service(
            ChromeDriverManager().install()), options=options)
        self.driver.implicitly_wait(0.5)  # has to be set only once
        print('Initialized successfully')

    def _create_product_catalog(self) -> dict:
        """clicks once through all product lines and products and saves them in a dict

        Returns:
            dict: product catalog
        """

        sel = Select(self.driver.find_element(
            By.CSS_SELECTOR, value=f"{SELECTOR_PRODUCT_TYPE}"))

        # set keys as product_lines
        product_catalog = dict.fromkeys(
            [elem.text for elem in sel.options[1:]], None)
        # set values from products of product line
        for product in product_catalog.keys():
            sel.select_by_visible_text(product)
            selector_products = Select(self.driver.find_element(
                By.XPATH, value=f"{PRODUCT_SELECTOR}"))
            product_catalog[product] = [
                elem.text for elem in selector_products.options[1:]]
        print('created product_catalog')
        return product_catalog

    def _choose_product_line(self, product_line: str) -> None:
        """selects product line on vendor website

        Args:
            product_line (str): product line to select
        """
        sel = Select(self.driver.find_element(By.CSS_SELECTOR,
                     value='div.margin_bottom20 > select:nth-child(1)'))

        sel.select_by_visible_text(product_line)
        selector_products = Select(self.driver.find_element(
            By.XPATH, value='//*[@id="heading_bg"]/div/div/div[2]/select'))
        # return [elem.text for elem in selector_products.options[1:]]

    def _choose_product(self, product: str) -> (str, str, str):
        """selects product on vendor website after selecting product line

        Args:
            product (str): product to select

        Returns:
            list[str]: MD5 checksum, list of available OS, drivers current url
        """
        self.driver.implicitly_wait(10)
        #time.sleep(1)
        # select product
        selector_products = Select(self.driver.find_element(
            By.XPATH, value=f'{PRODUCT_SELECTOR}'))
        selector_products.select_by_visible_text(product)
        # newest OS Version
        try:
            # wait up to 2 seconds to find sepcific element
            elem = WebDriverWait(self.driver, 2).until(
            EC.presence_of_element_located((By.XPATH, SELECTOR_NEWEST_OS)) 
            )
            selector_OS = self.driver.find_element(
                By.XPATH, value=f'{SELECTOR_NEWEST_OS}')
            # return MD5 checksum and DSM newest OS Version and current URL
            return self._get_MD5_checksum(), selector_OS.text, self.driver.current_url
        except Exception as e:
            print(f'Product {product} has no MD5 or OS ')
            # print(e)
            return None, None, None

    def download_product(self, product: str) -> bool:
        """
        """
        try:
            el = self.driver.find_element(
                By.XPATH, value=SELECTOR_DOWNLOAD)
            el.click()
            return True
        except Exception as e:
            print(e)
            return False

    def find_download_link(self, product_line: str, product: str) -> str:
        self._choose_product_line(product_line)
        self._choose_product(product)
        self.download_product(product)
        return self._get_MD5_checksum()

    def _get_MD5_checksum(self) -> str:
        try:
            el = self.driver.find_element(
                By.XPATH, value=SELECTOR_MD5)
            return el.get_attribute('title').replace('\n(Copy to Clipboard)', '')
        except Exception as e:
            print(e)
            return None

    def scrape_metadata(self) -> list[dict]:
        """function that gets executed from Core.py to scrape metadata 
            and search for firmwares

        Returns:
            list[dict]: list of dicts with metadata
        """
        # open website
        self.driver.get(self.url)
        # create product catalog
        self.product_catalog = self._create_product_catalog()
        # create list of dicts with metadata
        metadata = []
        for product_line in self.product_catalog.keys():
            self._choose_product_line(product_line)
            for product in tqdm(self.product_catalog[product_line]):
                md5, os, url = self._choose_product(product)
                metadata.append({'manufacturer': 'Synology',
                                 'product_type': product_line,
                                 'version': os,
                                 'product_name': product,
                                 'url': url,
                                 'checksum_scraped': md5,
                                 'download_link': 'not implemented'
                                 })
                if len(metadata) > self.max_products:
                    break
        self.driver.quit()
        return metadata

In [42]:
Syn = Synology_scraper()

Initialized successfully


In [43]:
Syn.scrape_metadata()

created product_catalog


 43%|████▎     | 12/28 [00:12<00:51,  3.22s/it]

Message: no such element: Unable to locate element: {"method":"xpath","selector":"//*[@id="results"]/div[3]/div[2]/div[1]/div/div[1]/div[1]/div/div/div[3]/div/div/div/div[2]/div[2]/div/a"}
  (Session info: chrome=108.0.5359.71)
Stacktrace:
0   chromedriver                        0x000000010d490f38 chromedriver + 4910904
1   chromedriver                        0x000000010d410a03 chromedriver + 4385283
2   chromedriver                        0x000000010d055747 chromedriver + 472903
3   chromedriver                        0x000000010d09a34c chromedriver + 754508
4   chromedriver                        0x000000010d09a5a1 chromedriver + 755105
5   chromedriver                        0x000000010d0dde94 chromedriver + 1031828
6   chromedriver                        0x000000010d0c013d chromedriver + 909629
7   chromedriver                        0x000000010d0db28e chromedriver + 1020558
8   chromedriver                        0x000000010d0bfee3 chromedriver + 909027
9   chromedriver           

 46%|████▋     | 13/28 [00:22<01:19,  5.32s/it]

Message: no such element: Unable to locate element: {"method":"xpath","selector":"//*[@id="results"]/div[3]/div[2]/div[1]/div/div[1]/div[1]/div/div/div[3]/div/div/div/div[2]/div[2]/div/a"}
  (Session info: chrome=108.0.5359.71)
Stacktrace:
0   chromedriver                        0x000000010d490f38 chromedriver + 4910904
1   chromedriver                        0x000000010d410a03 chromedriver + 4385283
2   chromedriver                        0x000000010d055747 chromedriver + 472903
3   chromedriver                        0x000000010d09a34c chromedriver + 754508
4   chromedriver                        0x000000010d09a5a1 chromedriver + 755105
5   chromedriver                        0x000000010d0dde94 chromedriver + 1031828
6   chromedriver                        0x000000010d0c013d chromedriver + 909629
7   chromedriver                        0x000000010d0db28e chromedriver + 1020558
8   chromedriver                        0x000000010d0bfee3 chromedriver + 909027
9   chromedriver           

...

Message: 
Stacktrace:
0   chromedriver                        0x000000010d490f38 chromedriver + 4910904
1   chromedriver                        0x000000010d410a03 chromedriver + 4385283
2   chromedriver                        0x000000010d055747 chromedriver + 472903
3   chromedriver                        0x000000010d09a34c chromedriver + 754508
4   chromedriver                        0x000000010d09a5a1 chromedriver + 755105
5   chromedriver                        0x000000010d0dde94 chromedriver + 1031828
6   chromedriver                        0x000000010d0c013d chromedriver + 909629
7   chromedriver                        0x000000010d0db28e chromedriver + 1020558
8   chromedriver                        0x000000010d0bfee3 chromedriver + 909027
9   chromedriver                        0x000000010d08a30c chromedriver + 688908
10  chromedriver                        0x000000010d08b88e chromedriver + 694414
11  chromedriver                        0x000000010d45e1de chromedriver + 4702686
1

 97%|█████████▋| 56/58 [06:02<00:21, 10.62s/it]

Message: 
Stacktrace:
0   chromedriver                        0x000000010d490f38 chromedriver + 4910904
1   chromedriver                        0x000000010d410a03 chromedriver + 4385283
2   chromedriver                        0x000000010d055747 chromedriver + 472903
3   chromedriver                        0x000000010d09a34c chromedriver + 754508
4   chromedriver                        0x000000010d09a5a1 chromedriver + 755105
5   chromedriver                        0x000000010d0dde94 chromedriver + 1031828
6   chromedriver                        0x000000010d0c013d chromedriver + 909629
7   chromedriver                        0x000000010d0db28e chromedriver + 1020558
8   chromedriver                        0x000000010d0bfee3 chromedriver + 909027
9   chromedriver                        0x000000010d08a30c chromedriver + 688908
10  chromedriver                        0x000000010d08b88e chromedriver + 694414
11  chromedriver                        0x000000010d45e1de chromedriver + 4702686
1

100%|██████████| 58/58 [06:05<00:00,  6.30s/it]


[{'manufacturer': 'Synology',
  'product_type': 'NAS',
  'version': 'DSM 7.1.1',
  'product_name': 'DS120j',
  'url': 'https://www.synology.com/en-global/support/download/DS120j?version=7.1#system',
  'checksum_scraped': '91d8b33559f051aa0a13080e5ecda61f',
  'download_link': 'not implemented'},
 {'manufacturer': 'Synology',
  'product_type': 'NAS',
  'version': 'DSM 7.1.1',
  'product_name': 'DS119j',
  'url': 'https://www.synology.com/en-global/support/download/DS119j?version=7.1#system',
  'checksum_scraped': '91d8b33559f051aa0a13080e5ecda61f',
  'download_link': 'not implemented'},
 {'manufacturer': 'Synology',
  'product_type': 'NAS',
  'version': 'DSM 7.1.1',
  'product_name': 'DS118',
  'url': 'https://www.synology.com/en-global/support/download/DS118?version=7.1#system',
  'checksum_scraped': '91d8b33559f051aa0a13080e5ecda61f',
  'download_link': 'not implemented'},
 {'manufacturer': 'Synology',
  'product_type': 'NAS',
  'version': 'DSM 7.1.1',
  'product_name': 'DS116',
  'url

In [44]:
Syn.product_catalog

{'NAS': ['DS120j',
  'DS119j',
  'DS118',
  'DS116',
  'DS115',
  'DS115j',
  'DS114',
  'DS112+',
  'DS112',
  'DS112j',
  'DS111',
  'DS110+',
  'DS110j',
  'DS109+',
  'DS109',
  'DS109j',
  'DS108j',
  'DS107+',
  'DS107',
  'DS107e',
  'DS-106',
  'DS-106j',
  'DS-106e',
  'DS-101g+',
  'DS-101',
  'DS-101j',
  'RS217',
  'RS214',
  'RS212',
  'DS720+',
  'DS718+',
  'DS716+II',
  'DS716+',
  'DS715',
  'DS713+',
  'DS712+',
  'DS710+',
  'DS220+',
  'DS220j',
  'DS218+',
  'DS218',
  'DS218play',
  'DS218j',
  'DS216+II',
  'DS216+',
  'DS216',
  'DS216play',
  'DS216j',
  'DS216se',
  'DS215+',
  'DS215j',
  'DS214+',
  'DS214',
  'DS214play',
  'DS214se',
  'DS213+',
  'DS213',
  'DS213j',
  'DS213air',
  'DS212+',
  'DS212',
  'DS212j',
  'DS211+',
  'DS211',
  'DS211j',
  'DS210+',
  'DS210j',
  'DS209+II',
  'DS209+',
  'DS209',
  'DS209j',
  'DS207+',
  'DS207',
  'ioSafe 220+',
  'ioSafe 218',
  'ioSafe 216',
  'ioSafe 214',
  'ioSafe 213',
  'RS1619xs+',
  'RS822RP+',
  '